#Sliver Layer

###Loading the data from the bronze_table(delta table) cleaning it and applying data validation on it

In [0]:
from pyspark.sql import SparkSession

# Table names
bronze_table = "bronze_sales"
silver_table = "silver_sales"
validated_table="validated_sales"

# Load Bronze data
df_bronze = spark.table(bronze_table)

# Show schema & sample
df_bronze.printSchema()
display(df_bronze.limit(5))

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- QUANTITYORDERED: integer (nullable = true)
 |-- PRICEEACH: double (nullable = true)
 |-- ORDERLINENUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- QTR_ID: integer (nullable = true)
 |-- MONTH_ID: integer (nullable = true)
 |-- YEAR_ID: integer (nullable = true)
 |-- PRODUCTLINE: string (nullable = true)
 |-- MSRP: integer (nullable = true)
 |-- PRODUCTCODE: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- ADDRESSLINE1: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TERRITORY: string (nullable = true)
 |-- CONTACTLASTNAME: string (nullable = true)
 |-- CONTACTFIRSTNAME: string (nullable = true)
 |-- DEALSIZE: string (nullable = true)


ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
10107,30,95.7,2,2871.0,2/24/2003 0:00,Shipped,1,2,2003,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,897 Long Airport Avenue,null,NYC,NY,10022,USA,NA,Yu,Kwai
10121,34,81.35,5,2765.9,5/7/2003 0:00,Shipped,2,5,2003,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,59 rue de l'Abbaye,null,Reims,null,51100,France,EMEA,Henriot,Paul
10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,null,Paris,null,75508,France,EMEA,Da Cunha,Daniel
10145,45,83.26,6,3746.7,8/25/2003 0:00,Shipped,3,8,2003,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,null,Pasadena,CA,90003,USA,NA,Young,Julie
10159,49,100.0,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,Motorcycles,95,S10_1678,Corporate Gift Ideas Co.,6505551386,7734 Strong St.,null,San Francisco,CA,null,USA,NA,Brown,Julie


In [0]:
from pyspark.sql.functions import expr

# Apply cleaning rules:
# 1. Drop nulls in critical columns (except ORDERDATE, we'll handle with try_to_timestamp)
# 2. Drop duplicates
# 3. Safely convert ORDERDATE to timestamp using try_to_timestamp

df_clean = (
    df_bronze.dropna(subset=["ORDERNUMBER", "SALES"])   # keep ORDERDATE for parsing
             .dropDuplicates()
             .withColumn("ORDERDATE", expr("try_to_timestamp(ORDERDATE, 'M/d/yyyy H:mm')"))
)

# Save as Silver table
spark.sql(f"DROP TABLE IF EXISTS {silver_table}")
df_clean.write.format("delta").mode("overwrite").saveAsTable(silver_table)

display(df_clean.limit(5))

ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
10262,40,84.03,4,3361.2,2004-06-24T00:00:00.000Z,Cancelled,2,6,2004,Planes,91,S700_1691,Euro Shopping Channel,(91) 555 94 44,"C/ Moralzarzal, 86",null,Madrid,null,28034,Spain,EMEA,Freyre,Diego
10193,32,79.37,5,2539.84,2003-11-21T00:00:00.000Z,Shipped,4,11,2003,Vintage Cars,99,S18_3320,"Australian Collectables, Ltd",61-9-3844-6555,7 Allen Street,null,Glen Waverly,Victoria,3150,Australia,APAC,Connery,Sean
10114,21,100.0,5,2925.09,2003-04-01T00:00:00.000Z,Shipped,2,4,2003,Trucks and Buses,127,S24_2300,"La Corne D'abondance, Co.",(1) 42.34.2555,"265, boulevard Charonne",null,Paris,null,75012,France,EMEA,Bertrand,Marie
10361,34,100.0,6,3871.92,2004-12-17T00:00:00.000Z,Shipped,4,12,2004,Vintage Cars,65,S24_3420,Souveniers And Things Co.,+61 2 9495 8555,"Monitor Money Building, 815 Pacific Hwy",Level 6,Chatswood,NSW,2067,Australia,APAC,Huxley,Adrian
10386,44,86.4,15,3801.6,2005-03-01T00:00:00.000Z,Resolved,1,3,2005,Planes,74,S700_4002,Euro Shopping Channel,(91) 555 94 44,"C/ Moralzarzal, 86",null,Madrid,null,28034,Spain,EMEA,Freyre,Diego


In [0]:
from pyspark.sql.functions import count, when, isnan, col

df_silver = spark.table(silver_table)

# Validation 1: Count rows
print("✅ Total rows in silver:", df_silver.count())

# Validation 2: Null check for important columns
df_silver.select([
    count(when(col(c).isNull() | (c == 'ORDERDATE'), c)).alias(c + "_nulls")
    for c in ["ORDERNUMBER", "SALES", "ORDERDATE"]
]).show()

# Validation 3: Check for negative sales
neg_sales = df_silver.filter(col("SALES") < 0).count()
print("⚠️ Rows with negative SALES:", neg_sales)

# Validation 4: Check date range sanity
df_silver.selectExpr(
    "min(ORDERDATE) as min_date", 
    "max(ORDERDATE) as max_date"
).show()

display(df_silver.limit(5))
df_silver.write.format("delta").mode("overwrite").saveAsTable(validated_table)

✅ Total rows in silver: 2823
+-----------------+-----------+---------------+
|ORDERNUMBER_nulls|SALES_nulls|ORDERDATE_nulls|
+-----------------+-----------+---------------+
|                0|          0|           2823|
+-----------------+-----------+---------------+

⚠️ Rows with negative SALES: 0
+-------------------+-------------------+
|           min_date|           max_date|
+-------------------+-------------------+
|2003-01-06 00:00:00|2005-05-31 00:00:00|
+-------------------+-------------------+



ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
10262,40,84.03,4,3361.2,2004-06-24T00:00:00.000Z,Cancelled,2,6,2004,Planes,91,S700_1691,Euro Shopping Channel,(91) 555 94 44,"C/ Moralzarzal, 86",null,Madrid,null,28034,Spain,EMEA,Freyre,Diego
10193,32,79.37,5,2539.84,2003-11-21T00:00:00.000Z,Shipped,4,11,2003,Vintage Cars,99,S18_3320,"Australian Collectables, Ltd",61-9-3844-6555,7 Allen Street,null,Glen Waverly,Victoria,3150,Australia,APAC,Connery,Sean
10114,21,100.0,5,2925.09,2003-04-01T00:00:00.000Z,Shipped,2,4,2003,Trucks and Buses,127,S24_2300,"La Corne D'abondance, Co.",(1) 42.34.2555,"265, boulevard Charonne",null,Paris,null,75012,France,EMEA,Bertrand,Marie
10361,34,100.0,6,3871.92,2004-12-17T00:00:00.000Z,Shipped,4,12,2004,Vintage Cars,65,S24_3420,Souveniers And Things Co.,+61 2 9495 8555,"Monitor Money Building, 815 Pacific Hwy",Level 6,Chatswood,NSW,2067,Australia,APAC,Huxley,Adrian
10386,44,86.4,15,3801.6,2005-03-01T00:00:00.000Z,Resolved,1,3,2005,Planes,74,S700_4002,Euro Shopping Channel,(91) 555 94 44,"C/ Moralzarzal, 86",null,Madrid,null,28034,Spain,EMEA,Freyre,Diego
